# *可选* 使用类分离逻辑

与之前的笔记本一样，这里的目标是将逻辑（根据给定长度生成字符字符串）与用户界面分离。

这次，我们创建一个类来表示用户界面，一个类来表示逻辑，还有第三个类来连接它们。

在这个过程中，我们将看到：

+ 一种新的方法，通过为数据赋予可以与小部件特征链接的特征，来将用户界面元素与数据连接起来。
+ 如何验证这些特征的值。
+ 如何正确地对子类化一个 `ipywidgets` 小部件。
+ 一种连接这两个类的方式。

In [ ]:
import ipywidgets as widgets
import traitlets

## 用户界面，作为一个类

这次我们将把密码小部件构建为 `VBox` 的子类。这样做的优点是使得它像 `ipywidgets` 的内建小部件一样，便于分发和使用。

我们将经历几个版本的迭代，最终的版本名为 `PassGenGUI`；在达到这个版本之前，我们会给每个版本添加版本号。

在这个第一个版本中，我们构建了与前两个笔记本中相同的界面。

代码中最容易忽视的一行是调用父类 `__init__` 的那一行。如果忘记这一行，虽然你已经从 `VBox` 子类化，但小部件将无法正常工作。

请注意，单个小部件是“私有的”，因为它们的名称前面加了下划线。当然，如果有你希望暴露给最终用户的元素，你可以公开它们。

In [ ]:
class PassGenGUI_incomplete(widgets.VBox):
    def __init__(self):
        # Do NOT forgot to do this ↓↓↓↓↓↓↓↓ 
        super(PassGenGUI_incomplete, self).__init__()
        
        # Define each of the children...
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('No password yet', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children

让我们来看看这个小部件…

In [ ]:
pwd_gen = PassGenGUI_incomplete()
pwd_gen

请注意，我们的小部件没有值；它确实应该有一个值，而且这个值应该是密码。虽然我们可以将密码作为一个 `traitlet` 添加，并将其链接到 `self._password_text` 的值，但我们将改为在类中添加一个只读属性。原因是我们不希望用户能够设置小部件的密码值。它的整个目的是为用户生成一个密码。

In [ ]:
class PassGenGUI(widgets.VBox):
    def __init__(self):
        # Do NOT forgot to do this!
        super(PassGenGUI, self).__init__()
        
        # Define each of the children...
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('No password yet', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})
        children = [self._helpful_title, self._password_text, self._password_length]
        self.children = children

    # Add value as a read-only property
    @property
    def value(self):
        return self._password_text.value

让我们试试看…

In [ ]:
pwd_gen2 = PassGenGUI()
pwd_gen2

确保你能获取到值...

In [ ]:
pwd_gen2.value

...并且确保你不能*设置*该值。

In [ ]:
pwd_gen2.value = 'new password'

## 逻辑类与可链接的属性

在前一版的密码生成器中，我们编写了一个函数来生成密码，另一个函数用于添加回调，即每当控件变化时调用该函数。这种方法没有问题，但当你添加更多控件时，它变得越来越复杂。

相反，我们在下面定义了一个类，它生成密码，并包含作为类属性的 `traitlets`，稍后我们将这些 `traitlets` 与控件 GUI 进行链接。这些 `traitlets` 类似于控件的键；它们可以链接到控件的键，逻辑类可以监视这些键的变化，并且可以设置这些键的值。

请注意，这个类中没有任何内容引用控件；它的逻辑完全是内部的。

为了包含 `traitlets`，类必须继承自 `traitlets.HasTraits`。

我们将分步骤实现这一点。首先，在下面的单元格中，我们创建 `PassGen` 类来表示密码生成逻辑，并定义一个名为 `length` 的属性，并定义一个方法，当 `length` 的值发生变化时会被调用。

In [ ]:
# Subclass from HasTraits
class PassGenLogic_v1(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    
    def __init__(self):
        # initialize the class by calling the superclass
        super(PassGenLogic_v1, self).__init__()
        
    # The observe decorator is used to indicate that the function being
    # decorated should be called if any of the traits listed as arguments
    # change. The decorated function MUST have an argument named change;
    # in this example the change variable isn't used because we want to update
    # the generated password no matter what the change was.
    
    # Note that we are monitoring changes in PassGen's **own trait**, length,
    # rather than a change in one of the controls in the GUI.
    
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import secrets
    
        new_password = ''.join(secrets.choice(string.ascii_letters) for _ in range(self.length))
        
        # We don't return anything (explanation later). For now just print the 
        # password to observe what happens when we change the length.
        print('In password generator password is: ', new_password)

我们将创建该类的一个实例，以便查看当我们更改 `length` 属性时会发生什么。

In [ ]:
p = PassGenLogic_v1()

尝试在下面的单元格中更改 `length` 属性。请注意，每次更改时，都会生成一个新的密码。

In [ ]:
p.length = 20

### 添加一个持有密码值的特性

我们实际上不希望每次生成新密码时都打印出来，当然，最终我们希望将其连接到基于小部件的GUI。下面的类添加了一个新的trait来保存生成的密码，并在`calculate_password`中设置该trait。你还可以返回生成的密码；例如，在测试你的模型代码时，这可能会很有用。

In [ ]:
# Subclass from HasTraits
class PassGenLogic_v2(traitlets.HasTraits):
    # Define your traits here, as class attributes 
    length = traitlets.Integer()
    
    # The new trait is defined here:
    password = traitlets.Unicode()
    
    def __init__(self):
        super(PassGenLogic_v2, self).__init__()
            
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import secrets
    
        new_password = ''.join(secrets.choice(string.ascii_letters) for _ in range(self.length))
        
        # Set the value of the password trait here:
        self.password = new_password

现在我们来试试：

In [ ]:
p2 = PassGenLogic_v2()

In [ ]:
p2.length = 110
p2.password

请注意，目前的代码并没有阻止设置一个不合理的长度：

In [ ]:
p2.length = -14
p2.password

### 添加密码长度的验证

Traitlets 提供了一种机制，用于通过 `@validate` 装饰器验证值。有关验证的文档请参阅 [这里](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#validation)，但下面的代码已经展示了核心思想。

这是最终版本的 `PassGen`，因此我们去掉了版本号。

只有新增的行会被注释。

In [ ]:
class PassGenLogic(traitlets.HasTraits):
    length = traitlets.Integer()
    password = traitlets.Unicode()
    
    def __init__(self):
        super(PassGenLogic, self).__init__()
            
    @traitlets.observe('length')
    def calculate_password(self, change):
        import string
        import secrets
    
        new_password = ''.join(secrets.choice(string.ascii_letters) for _ in range(self.length))
        
        # Set the value of the password trait here:
        self.password = new_password

    # The new validator:
    @traitlets.validate('length')
    # You can name this method anything you want.
    def _validate_length(self, proposal):
        # proposal contains the new value
        length = proposal['value']
        
        # Test the value
        if length < 1:
            # if it is a bad value, raise a TraitError
            raise traitlets.TraitError('Password length should be positive.')
        # If the value is good, return it.
        return proposal['value']

In [ ]:
p3 = PassGenLogic()

尝试在下面的单元格中设置负值作为密码长度：

In [ ]:
p3.length = -23

## 将各部分整合在一起

现在我们有了界面 `PassGenGUI` 和用于封装计算的类 `PassGenLogic`，接下来需要将它们连接起来。

我们将通过创建第三个类 `PassGen` 来实现连接。这并不是唯一的做法——例如，下面的代码也可以直接包含在 `PassGenGUI` 中。

当然，也可以将控制代码完全独立到一个单独的类中。无论控制代码在哪个类中，用户应该导入的是那个类。

这是最终版的 `PassGenUI`，因此我们去掉了版本号。

In [ ]:
# By subclassing from PassGenGUI we ensure PassGen behaves as a widget, since
# PassGenGUI subclasses from ipywidgets.VBox

class PassGen(PassGenGUI):
    def __init__(self):
        # Set up all of the subwidgets like _passwordd)text
        super(PassGen, self).__init__()
        
        # Create an instance of our model.
        self.model = PassGenLogic()
        
        # Link the password from the model to the _password_text widget
        traitlets.link((self.model, 'password'), (self._password_text, 'value'))
        
        # Link the _password_length widget to the length in the model
        traitlets.link((self.model, 'length'), (self._password_length, 'value'))

让我们创建该类的实例，显示它，并试试看！

In [ ]:
password_widget = PassGen()
password_widget